# Lab-3 Assignment

**Course:** CSL304 AI  
**Name:** Saurav Gupta  
**ID:** 12341940

## Minimum Cost Hamiltonian Cycle using MST Heuristics

In [14]:
import heapq
import math

def mst_cost(g, verts):
    if not verts:
        return 0
    s = next(iter(verts))
    used = {s}
    edges = [(w, s, nb) for nb, w in g[s] if nb in verts]
    heapq.heapify(edges)
    total = 0
    while edges and len(used) < len(verts):
        w, u, v = heapq.heappop(edges)
        if v in used:
            continue
        used.add(v)
        total += w
        for nb, w2 in g[v]:
            if nb in verts and nb not in used:
                heapq.heappush(edges, (w2, v, nb))
    return total if len(used) == len(verts) else math.inf

def ham_astar(g, start):
    verts = list(g.keys())
    pos = {v: i for i, v in enumerate(verts)}
    n = len(verts)

    def bit(v): return 1 << pos[v]

    full = (1 << n) - 1
    init = bit(start)
    pq = [(0, 0, start, init, [start])]
    seen = {}

    while pq:
        f, cost, u, mask, path = heapq.heappop(pq)
        if mask == full:
            for nb, w in g[u]:
                if nb == start:
                    return path + [start]
            continue
        if (u, mask) in seen and seen[(u, mask)] <= cost:
            continue
        seen[(u, mask)] = cost
        for nb, w in g[u]:
            if mask & bit(nb):
                continue
            nm = mask | bit(nb)
            nc = cost + w
            rem = [v for v in verts if not (nm & bit(v))]
            h = mst_cost(g, set(rem) | {start, nb})
            heapq.heappush(pq, (nc + h, nc, nb, nm, path + [nb]))
    return None

graph = {
    0: [(1, 10), (2, 15), (3, 20)],
    1: [(0, 10), (2, 35), (3, 25)],
    2: [(0, 15), (1, 35), (3, 30)],
    3: [(0, 20), (1, 25), (2, 30)]
}

res = ham_astar(graph, 0)

if res:
    total = 0
    for i in range(len(res) - 1):
        u, v = res[i], res[i + 1]
        for nb, w in graph[u]:
            if nb == v:
                total += w
                break
    print("Cycle:", res)
    print("Cost:", total)
else:
    print("No cycle found")


Cycle: [0, 1, 3, 2, 0]
Cost: 80


## Minimum Cost Hamiltonian Cycle using Branch and Bound

In [15]:
import math

def ham_bb(g, start):
    n = len(g)
    best_cost = [math.inf]
    best_path = [[]]

    low = [min(w for nb, w in g[u]) for u in range(n)]

    def explore(route, seen, c):
        u = route[-1]
        bound = c + sum(low[v] for v in range(n) if v not in seen)
        if bound >= best_cost[0]:
            return
        if len(route) == n:
            for nb, w in g[u]:
                if nb == start:
                    total = c + w
                    if total < best_cost[0]:
                        best_cost[0] = total
                        best_path[0] = route + [start]
            return
        for nb, w in g[u]:
            if nb not in seen:
                explore(route + [nb], seen | {nb}, c + w)

    explore([start], {start}, 0)
    return (best_path[0], best_cost[0]) if best_cost[0] != math.inf else (None, math.inf)


graph = {
    0: [(1, 10), (2, 15), (3, 20)],
    1: [(0, 10), (2, 35), (3, 25)],
    2: [(0, 15), (1, 35), (3, 30)],
    3: [(0, 20), (1, 25), (2, 30)]
}

path, cost = ham_bb(graph, 0)

if path:
    print("Cycle:", path)
    print("Cost:", cost)
else:
    print("No cycle found")


Cycle: [0, 1, 3, 2, 0]
Cost: 80
